In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [2]:
#Please change to your name.
name = "Michael Harris"

#Please use this to control EDA. 
show_eda = False

In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
41,0,0.881,0.425,0.028,0.451,0.673,0.967,0.895,0.622,0.092,...,0.328,0.589,0.889,0.177,0.439,0.213,0.400,0.753,0.643,0.062
22,0,0.311,0.861,0.040,0.959,0.021,0.815,0.176,0.799,0.001,...,0.628,0.757,0.686,0.390,0.496,0.461,0.900,0.017,0.053,0.727
213,1,0.698,0.456,0.489,0.410,0.771,0.750,0.317,0.043,0.775,...,0.914,0.829,0.176,0.300,0.123,0.003,0.506,0.011,0.139,0.758
130,0,0.782,0.213,0.691,0.877,0.645,0.800,0.650,0.094,0.966,...,0.168,0.167,0.368,0.608,0.050,0.171,0.650,0.125,0.656,0.134
5,0,0.961,0.910,0.399,0.860,0.883,0.032,0.618,0.694,0.155,...,0.027,0.296,0.362,0.983,0.348,0.509,0.447,0.803,0.911,0.998


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [34]:
df2 = pd.get_dummies(df, drop_first=True)
y = np.array(df2["target"]).reshape(-1,1)
X = np.array(df2.drop(columns={"target"}))
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler()
estimator = RandomForestClassifier(n_jobs=-1)
pipe = Pipeline(steps=[("scaler", scaler), ("forest", estimator)])

params = {'forest__max_depth':[5,6,7,8,9],
          'forest__min_samples_split':[3,4,5,6,7],
          'forest__min_samples_leaf' :[2,3,4,5,6],
          'forest__n_estimators' :[20,50,80]}
 
best = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1) 
best.fit(X_train, y_train.ravel())
best1 = best.best_estimator_
print(best1.score(X_test, y_test))
print(best1)

0.5555555555555556
Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(max_depth=5, min_samples_leaf=2,
                                        min_samples_split=4, n_estimators=80,
                                        n_jobs=-1))])


In [36]:
clf = DecisionTreeClassifier(max_depth=20,min_samples_split=5, min_samples_leaf=2)
clf = clf.fit(X_train, y_train)
print(clf.get_depth())
print("Training Accuracy:", clf.score(X_train, y_train))
print("Testing Accuracy:", clf.score(X_test, y_test))

7
Training Accuracy: 0.9625668449197861
Testing Accuracy: 0.5396825396825397


In [51]:
#Build pipeline
pipeline_steps = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(max_depth=20,min_samples_split=5, min_samples_leaf=2))
    ]

best2 = Pipeline(pipeline_steps)

best2.fit(X_train, y_train)

print(best2.score(X_train, y_train))
print(best2.score(X_test, y_test))
   

0.9625668449197861
0.5396825396825397


### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [50]:
print(best.score(X_test, y_test))
print(best)

0.5555555555555556
GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('forest',
                                        RandomForestClassifier(n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'forest__max_depth': [5, 6, 7, 8, 9],
                         'forest__min_samples_leaf': [2, 3, 4, 5, 6],
                         'forest__min_samples_split': [3, 4, 5, 6, 7],
                         'forest__n_estimators': [20, 50, 80]})


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [46]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.6120335075608113
0.611493670886076
Michael Harris 0.6117635892234436


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li> For the model I used a forest instead of a single tree, in combination with a scalar
<li> For that forest used a grid search with a combination of four parameters
</ul>